In [1]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath('src'))
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [2]:
# Load packages
import pandas as pd
import numpy as np

import sklearn.metrics as skm
import sklearn.preprocessing as skp
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from src.utils import MlflowUtils
from sklearn.pipeline import make_pipeline
from modelingUtlis import ModelingUtils
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff


In [3]:
LOAD_PATH = '../data/interim/'
LOAD_SKILLS_DEV = '6.0-Engineered_data-split_roles-cluster_skills.pkl'
SAVE_DF_NAME = '7.0-Chosen_features_and_roles.pkl'
SAVE_DF_NAME_CSV = '7.0-Chosen_features_and_roles.csv'

# Load Data

In [4]:
skills_dev_df = pd.read_pickle(LOAD_PATH + LOAD_SKILLS_DEV)

In [5]:
skills_dev_df

Languages                                                             \
            APL Assembly Bash/Shell    C    C#   C++ COBOL Clojure Crystal   
2           0.0      0.0        0.0  0.0  0.75  0.75   0.0     0.0     0.0   
3           0.0      0.0        0.0  0.0  1.50  0.00   0.0     0.0     0.0   
7           0.5      0.0        0.5  1.0  0.00  1.00   0.0     0.0     0.0   
9           0.0      0.0        0.0  0.0  0.00  0.00   0.0     0.0     0.0   
10          0.0      0.0        1.0  0.0  1.50  0.00   0.0     0.0     0.0   
...         ...      ...        ...  ...   ...   ...   ...     ...     ...   
73262       0.0      0.5        0.0  0.5  0.00  0.75   0.0     0.0     0.0   
73263       0.0      0.0        1.5  0.0  0.00  0.00   0.0     0.0     0.0   
73264       0.0      0.0        1.0  0.0  0.00  0.00   0.0     0.0     0.0   
73265       0.0      0.0        0.0  0.0  0.25  0.00   0.0     0.0     0.0   
73266       0.0      0.0        0.0  0.0  1.00  0.00   0.0     0.0     0.0   

            ...         DevType                                        \
      Dart  ... full_stack_Java full_stack_.JavaScript full_stack_PHP   
2      0.0  ...               0                      0              0   
3      0.0  ...               0                      0              0   
7      0.0  ...               1                      1              1   
9      0.0  ...               0                      0              0   
10     0.0  ...               0                      0              0   
...    ...  ...             ...                    ...            ...   
73262  0.0  ...               0                      0              0   
73263  1.0  ...               0                      0              0   
73264  0.0  ...               0                      0              0   
73265  0.0  ...               0                      0              1   
73266  0.0  ...               0                      0              0   

                                                                          \
      full_stack_python back_end_Java back_end_.JavaScript back_end_.NET   
2                     0             0                    0             0   
3                     0             0                    0             0   
7                     0             0                    0             0   
9                     0             0                    0             0   
10                    0             0                    0             1   
...                 ...           ...                  ...           ...   
73262                 0             0                    0             0   
73263                 0             0                    1             0   
73264                 0             0                    0             0   
73265                 1             0                    0             0   
73266                 0             0                    0             0   

                                                 
      back_end_C++ back_end_python back_end_PHP  
2                0               0            0  
3                0               0            0  
7                0               0            0  
9                0               0            0  
10               0               0            0  
...            ...             ...          ...  
73262            0               0            0  
73263            1               1            1  
73264            0               0            0  
73265            0               0            0  
73266            0               0            0  

[50423 rows x 228 columns]

# 1. Choosing features
we need to choose between original features or clusters features, which one gives us the best results in classification to proceed with them in hero model

we will use class weights formula:
# $$ w_j=\frac{n\_samples}{(n\_classes * n\_samples_j)} $$

In [8]:
basic_model = make_pipeline(skp.StandardScaler(), MultiOutputClassifier(LogisticRegression()))


In [9]:
original_df = skills_dev_df.drop('clustered_skills', level = 0, axis = 1)
original_modeling_utils = ModelingUtils(original_df, basic_model)
original_features_model, classification_report_original_features = original_modeling_utils.train_evaluate_model_features()

train_f1-score:  0.3960388235971532
test_f1-score:  0.39148719761665596


In [10]:
drop_columns = original_df.drop('DevType', level=0, axis=1).columns
clustered_df = skills_dev_df.drop(drop_columns, axis = 1)
clustered_modeling_utils = ModelingUtils(clustered_df, basic_model)
clustered_features_model, classification_report_clustered_features = clustered_modeling_utils.train_evaluate_model_features()

train_f1-score:  0.3203918078121181
test_f1-score:  0.3183727014803502


In [11]:
classification_report_original_features.loc['Mean',:] = classification_report_original_features.mean()
classification_report_original_features

train                      \
                                              precision    recall  f1_score   
Academic researcher                            0.451994  0.223848  0.299413   
Blockchain                                     0.432836  0.262206  0.326577   
Cloud infrastructure engineer                  0.429658  0.215958  0.287440   
Data or business analyst                       0.393064  0.113617  0.176280   
Data scientist or machine learning specialist  0.642804  0.498569  0.561573   
Database administrator                         0.284404  0.027169  0.049600   
DevOps specialist                              0.502580  0.192642  0.278524   
Developer_QA or test                           0.333333  0.000968  0.001931   
Developer_back-end                             0.592902  0.621670  0.606946   
Developer_desktop or enterprise applications   0.600000  0.175177  0.271180   
Developer_embedded applications or devices     0.594223  0.260084  0.361809   
Developer_front-end                            0.612361  0.381277  0.469948   
Developer_full-stack                           0.674467  0.732796  0.702423   
Developer_game or graphics                     0.431481  0.315718  0.364632   
Developer_mobile                               0.742877  0.585853  0.655087   
Engineer_data                                  0.535235  0.201643  0.292929   
Engineer_site reliability                      0.318841  0.133942  0.188639   
Scientist                                      0.395712  0.255990  0.310873   
Security professional                          0.207317  0.029160  0.051128   
Senior Executive (C-Suite_VP_etc.)             0.357143  0.008078  0.015798   
System administrator                           0.300000  0.048832  0.083993   
full_stack_.NET                                0.699579  0.634383  0.665388   
full_stack_Java                                0.640811  0.560942  0.598222   
full_stack_.JavaScript                         0.685138  0.702110  0.693520   
full_stack_PHP                                 0.690861  0.641641  0.665342   
full_stack_python                              0.648820  0.546132  0.593064   
back_end_Java                                  0.601261  0.610846  0.606016   
back_end_.JavaScript                           0.452622  0.466810  0.459607   
back_end_.NET                                  0.518011  0.553708  0.535265   
back_end_C++                                   0.552706  0.443727  0.492257   
back_end_python                                0.565906  0.453192  0.503316   
back_end_PHP                                   0.527819  0.483199  0.504525   
Mean                                           0.513024  0.355684  0.396039   

                                                             test            \
                                               accuracy precision    recall   
Academic researcher                            0.964498  0.452941  0.236923   
Blockchain                                     0.985175  0.485714  0.242857   
Cloud infrastructure engineer                  0.944442  0.386100  0.189753   
Data or business analyst                       0.968490  0.434343  0.134375   
Data scientist or machine learning specialist  0.966283  0.613941  0.494600   
Database administrator                         0.970547  0.269231  0.026515   
DevOps specialist                              0.937450  0.469027  0.172638   
Developer_QA or test                           0.974365  0.500000  0.003788   
Developer_back-end                             0.701730  0.597698  0.609333   
Developer_desktop or enterprise applications   0.898056  0.595808  0.185116   
Developer_embedded applications or devices     0.962217  0.580488  0.288835   
Developer_front-end                            0.830995  0.593042  0.373598   
Developer_full-stack                           0.742091  0.674813  0.734484   
Developer_game or graphics                     0.979869  0.482014  0.323671   
Developer_mobile    

In [12]:
classification_report_clustered_features.loc['Mean',:] = classification_report_clustered_features.mean()
classification_report_clustered_features

train                      \
                                              precision    recall  f1_score   
Academic researcher                            0.389549  0.121033  0.184685   
Blockchain                                     0.177215  0.026316  0.045827   
Cloud infrastructure engineer                  0.386454  0.139035  0.204498   
Data or business analyst                       0.375587  0.065681  0.111810   
Data scientist or machine learning specialist  0.636295  0.472067  0.542014   
Database administrator                         0.285714  0.001794  0.003565   
DevOps specialist                              0.453039  0.065365  0.114246   
Developer_QA or test                           0.000000  0.000000  0.000000   
Developer_back-end                             0.573565  0.569994  0.571774   
Developer_desktop or enterprise applications   0.609921  0.124482  0.206765   
Developer_embedded applications or devices     0.552106  0.150000  0.235907   
Developer_front-end                            0.597786  0.328143  0.423702   
Developer_full-stack                           0.668817  0.704074  0.685993   
Developer_game or graphics                     0.404063  0.231865  0.294650   
Developer_mobile                               0.744722  0.521346  0.613329   
Engineer_data                                  0.503464  0.138589  0.217348   
Engineer_site reliability                      0.277778  0.055118  0.091984   
Scientist                                      0.351906  0.149440  0.209790   
Security professional                          0.333333  0.005000  0.009852   
Senior Executive (C-Suite_VP_etc.)             0.000000  0.000000  0.000000   
System administrator                           0.100000  0.002835  0.005513   
full_stack_.NET                                0.705781  0.585238  0.639882   
full_stack_Java                                0.615692  0.472934  0.534952   
full_stack_.JavaScript                         0.687187  0.676027  0.681561   
full_stack_PHP                                 0.675497  0.560880  0.612876   
full_stack_python                              0.601968  0.358806  0.449616   
back_end_Java                                  0.603400  0.546406  0.573490   
back_end_.JavaScript                           0.393729  0.342052  0.366076   
back_end_.NET                                  0.501960  0.479093  0.490260   
back_end_C++                                   0.484098  0.252841  0.332185   
back_end_python                                0.520930  0.296820  0.378165   
back_end_PHP                                   0.510009  0.357317  0.420222   
Mean                                           0.460049  0.275018  0.320392   

                                                             test            \
                                               accuracy precision    recall   
Academic researcher                            0.964130  0.460000  0.136499   
Blockchain                                     0.985558  0.150000  0.018634   
Cloud infrastructure engineer                  0.943916  0.333333  0.127135   
Data or business analyst                       0.968515  0.388889  0.070234   
Data scientist or machine learning specialist  0.964625  0.585139  0.450000   
Database administrator                         0.972305  0.333333  0.003448   
DevOps specialist                              0.937005  0.450980  0.072670   
Developer_QA or test                           0.974039  0.000000  0.000000   
Developer_back-end                             0.684081  0.583893  0.579228   
Developer_desktop or enterprise applications   0.897171  0.636771  0.129562   
Developer_embedded applications or devices     0.960043  0.568807  0.150121   
Developer_front-end                            0.825332  0.601869  0.323781   
Developer_full-stack                           0.731941  0.656908  0.693720   
Developer_game or graphics                     0.978770  0.437500  0.242775   
Developer_mobile    


## 1. From the above results we will choose original features as our features
## 2. we need to also Exclude roles that have f1-score less than 0.1

In [13]:
drop_roles = classification_report_original_features[classification_report_original_features['test']['recall'] < .1].index
drop_roles

Index(['Database administrator', 'Developer_QA or test',
       'Security professional', 'Senior Executive (C-Suite_VP_etc.)',
       'System administrator'],
      dtype='object')

In [14]:
classification_report_original_features = classification_report_original_features.drop(drop_roles)
classification_report_original_features.loc['Mean',:] = classification_report_original_features.mean()
classification_report_original_features

train                      \
                                              precision    recall  f1_score   
Academic researcher                            0.451994  0.223848  0.299413   
Blockchain                                     0.432836  0.262206  0.326577   
Cloud infrastructure engineer                  0.429658  0.215958  0.287440   
Data or business analyst                       0.393064  0.113617  0.176280   
Data scientist or machine learning specialist  0.642804  0.498569  0.561573   
DevOps specialist                              0.502580  0.192642  0.278524   
Developer_back-end                             0.592902  0.621670  0.606946   
Developer_desktop or enterprise applications   0.600000  0.175177  0.271180   
Developer_embedded applications or devices     0.594223  0.260084  0.361809   
Developer_front-end                            0.612361  0.381277  0.469948   
Developer_full-stack                           0.674467  0.732796  0.702423   
Developer_game or graphics                     0.431481  0.315718  0.364632   
Developer_mobile                               0.742877  0.585853  0.655087   
Engineer_data                                  0.535235  0.201643  0.292929   
Engineer_site reliability                      0.318841  0.133942  0.188639   
Scientist                                      0.395712  0.255990  0.310873   
full_stack_.NET                                0.699579  0.634383  0.665388   
full_stack_Java                                0.640811  0.560942  0.598222   
full_stack_.JavaScript                         0.685138  0.702110  0.693520   
full_stack_PHP                                 0.690861  0.641641  0.665342   
full_stack_python                              0.648820  0.546132  0.593064   
back_end_Java                                  0.601261  0.610846  0.606016   
back_end_.JavaScript                           0.452622  0.466810  0.459607   
back_end_.NET                                  0.518011  0.553708  0.535265   
back_end_C++                                   0.552706  0.443727  0.492257   
back_end_python                                0.565906  0.453192  0.503316   
back_end_PHP                                   0.527819  0.483199  0.504525   
Mean                                           0.551700  0.415120  0.459530   

                                                             test            \
                                               accuracy precision    recall   
Academic researcher                            0.964498  0.452941  0.236923   
Blockchain                                     0.985175  0.485714  0.242857   
Cloud infrastructure engineer                  0.944442  0.386100  0.189753   
Data or business analyst                       0.968490  0.434343  0.134375   
Data scientist or machine learning specialist  0.966283  0.613941  0.494600   
DevOps specialist                              0.937450  0.469027  0.172638   
Developer_back-end                             0.701730  0.597698  0.609333   
Developer_desktop or enterprise applications   0.898056  0.595808  0.185116   
Developer_embedded applications or devices     0.962217  0.580488  0.288835   
Developer_front-end                            0.830995  0.593042  0.373598   
Developer_full-stack                           0.742091  0.674813  0.734484   
Developer_game or graphics                     0.979869  0.482014  0.323671   
Developer_mobile                               0.944863  0.721289  0.565313   
Engineer_data                                  0.961821  0.509554  0.200000   
Engineer_site reliability                      0.981233  0.343750  0.137500   
Scientist                                      0.977687  0.347107  0.200957   
full_stack_.NET                                0.900536  0.687679  0.628272   
full_stack_Java                                0.877851  0.622285  0.548487   
full_stack_.JavaScript                         0.853431  0.691322  0.700000   
full_stack_PHP      

In [15]:
classification_report_clustered_features = classification_report_clustered_features.drop(drop_roles)
classification_report_clustered_features.loc['Mean',:] = classification_report_clustered_features.mean()
classification_report_clustered_features

train                      \
                                              precision    recall  f1_score   
Academic researcher                            0.389549  0.121033  0.184685   
Blockchain                                     0.177215  0.026316  0.045827   
Cloud infrastructure engineer                  0.386454  0.139035  0.204498   
Data or business analyst                       0.375587  0.065681  0.111810   
Data scientist or machine learning specialist  0.636295  0.472067  0.542014   
DevOps specialist                              0.453039  0.065365  0.114246   
Developer_back-end                             0.573565  0.569994  0.571774   
Developer_desktop or enterprise applications   0.609921  0.124482  0.206765   
Developer_embedded applications or devices     0.552106  0.150000  0.235907   
Developer_front-end                            0.597786  0.328143  0.423702   
Developer_full-stack                           0.668817  0.704074  0.685993   
Developer_game or graphics                     0.404063  0.231865  0.294650   
Developer_mobile                               0.744722  0.521346  0.613329   
Engineer_data                                  0.503464  0.138589  0.217348   
Engineer_site reliability                      0.277778  0.055118  0.091984   
Scientist                                      0.351906  0.149440  0.209790   
full_stack_.NET                                0.705781  0.585238  0.639882   
full_stack_Java                                0.615692  0.472934  0.534952   
full_stack_.JavaScript                         0.687187  0.676027  0.681561   
full_stack_PHP                                 0.675497  0.560880  0.612876   
full_stack_python                              0.601968  0.358806  0.449616   
back_end_Java                                  0.603400  0.546406  0.573490   
back_end_.JavaScript                           0.393729  0.342052  0.366076   
back_end_.NET                                  0.501960  0.479093  0.490260   
back_end_C++                                   0.484098  0.252841  0.332185   
back_end_python                                0.520930  0.296820  0.378165   
back_end_PHP                                   0.510009  0.357317  0.420222   
Mean                                           0.516520  0.323785  0.376929   

                                                             test            \
                                               accuracy precision    recall   
Academic researcher                            0.964130  0.460000  0.136499   
Blockchain                                     0.985558  0.150000  0.018634   
Cloud infrastructure engineer                  0.943916  0.333333  0.127135   
Data or business analyst                       0.968515  0.388889  0.070234   
Data scientist or machine learning specialist  0.964625  0.585139  0.450000   
DevOps specialist                              0.937005  0.450980  0.072670   
Developer_back-end                             0.684081  0.583893  0.579228   
Developer_desktop or enterprise applications   0.897171  0.636771  0.129562   
Developer_embedded applications or devices     0.960043  0.568807  0.150121   
Developer_front-end                            0.825332  0.601869  0.323781   
Developer_full-stack                           0.731941  0.656908  0.693720   
Developer_game or graphics                     0.978770  0.437500  0.242775   
Developer_mobile                               0.940497  0.739425  0.506961   
Engineer_data                                  0.961108  0.556701  0.132029   
Engineer_site reliability                      0.982882  0.300000  0.049451   
Scientist                                      0.977606  0.389474  0.185930   
full_stack_.NET                                0.897196  0.712800  0.587731   
full_stack_Java                                0.867172  0.605049  0.453879   
full_stack_.JavaScript                         0.850649  0.656957  0.636747   
full_stack_PHP      

In [16]:
job_names = classification_report_clustered_features[:-1].index
job_names = list(job_names)
job_names

['Academic researcher',
 'Blockchain',
 'Cloud infrastructure engineer',
 'Data or business analyst',
 'Data scientist or machine learning specialist',
 'DevOps specialist',
 'Developer_back-end',
 'Developer_desktop or enterprise applications',
 'Developer_embedded applications or devices',
 'Developer_front-end',
 'Developer_full-stack',
 'Developer_game or graphics',
 'Developer_mobile',
 'Engineer_data',
 'Engineer_site reliability',
 'Scientist',
 'full_stack_.NET',
 'full_stack_Java',
 'full_stack_.JavaScript',
 'full_stack_PHP',
 'full_stack_python',
 'back_end_Java',
 'back_end_.JavaScript',
 'back_end_.NET',
 'back_end_C++',
 'back_end_python',
 'back_end_PHP']

# 6. Concat skills_df with DevType

In [17]:
skills_df = skills_dev_df.drop('clustered_skills', axis = 1, level=0).drop('DevType', axis = 1, level = 0)

In [18]:
dev_df = skills_dev_df['DevType'].drop(drop_roles, axis = 1)
dev_df.columns = pd.MultiIndex.from_product([['DevType'],job_names])


In [19]:
skills_dev_df = pd.concat([skills_df, dev_df], axis=1, join='inner')
skills_dev_df

Languages                                                             \
            APL Assembly Bash/Shell    C    C#   C++ COBOL Clojure Crystal   
2           0.0      0.0        0.0  0.0  0.75  0.75   0.0     0.0     0.0   
3           0.0      0.0        0.0  0.0  1.50  0.00   0.0     0.0     0.0   
7           0.5      0.0        0.5  1.0  0.00  1.00   0.0     0.0     0.0   
9           0.0      0.0        0.0  0.0  0.00  0.00   0.0     0.0     0.0   
10          0.0      0.0        1.0  0.0  1.50  0.00   0.0     0.0     0.0   
...         ...      ...        ...  ...   ...   ...   ...     ...     ...   
73262       0.0      0.5        0.0  0.5  0.00  0.75   0.0     0.0     0.0   
73263       0.0      0.0        1.5  0.0  0.00  0.00   0.0     0.0     0.0   
73264       0.0      0.0        1.0  0.0  0.00  0.00   0.0     0.0     0.0   
73265       0.0      0.0        0.0  0.0  0.25  0.00   0.0     0.0     0.0   
73266       0.0      0.0        0.0  0.0  1.00  0.00   0.0     0.0     0.0   

            ...         DevType                                        \
      Dart  ... full_stack_Java full_stack_.JavaScript full_stack_PHP   
2      0.0  ...               0                      0              0   
3      0.0  ...               0                      0              0   
7      0.0  ...               1                      1              1   
9      0.0  ...               0                      0              0   
10     0.0  ...               0                      0              0   
...    ...  ...             ...                    ...            ...   
73262  0.0  ...               0                      0              0   
73263  1.0  ...               0                      0              0   
73264  0.0  ...               0                      0              0   
73265  0.0  ...               0                      0              1   
73266  0.0  ...               0                      0              0   

                                                                          \
      full_stack_python back_end_Java back_end_.JavaScript back_end_.NET   
2                     0             0                    0             0   
3                     0             0                    0             0   
7                     0             0                    0             0   
9                     0             0                    0             0   
10                    0             0                    0             1   
...                 ...           ...                  ...           ...   
73262                 0             0                    0             0   
73263                 0             0                    1             0   
73264                 0             0                    0             0   
73265                 1             0                    0             0   
73266                 0             0                    0             0   

                                                 
      back_end_C++ back_end_python back_end_PHP  
2                0               0            0  
3                0               0            0  
7                0               0            0  
9                0               0            0  
10               0               0            0  
...            ...             ...          ...  
73262            0               0            0  
73263            1               1            1  
73264            0               0            0  
73265            0               0            0  
73266            0               0            0  

[50423 rows x 188 columns]

In [18]:
skills_dev_df.to_pickle(LOAD_PATH + SAVE_DF_NAME)


In [20]:
skills_dev_df.to_csv(LOAD_PATH + SAVE_DF_NAME_CSV)

# Save logs

## 1. original features

In [19]:
classification_report_original_features

train                      \
                                              precision    recall  f1_score   
Academic researcher                            0.462963  0.220426  0.298656   
Blockchain                                     0.440729  0.272556  0.336818   
Cloud infrastructure engineer                  0.431099  0.217780  0.289375   
Data or business analyst                       0.405914  0.124485  0.190536   
Data scientist or machine learning specialist  0.638930  0.485601  0.551813   
DevOps specialist                              0.504124  0.193357  0.279508   
Developer_back-end                             0.596681  0.621157  0.608673   
Developer_desktop or enterprise applications   0.591538  0.166781  0.260200   
Developer_embedded applications or devices     0.592643  0.265406  0.366625   
Developer_front-end                            0.617082  0.387913  0.476369   
Developer_full-stack                           0.674350  0.731881  0.701939   
Developer_game or graphics                     0.471136  0.338688  0.394081   
Developer_mobile                               0.740981  0.570714  0.644797   
Engineer_data                                  0.519169  0.202998  0.291872   
Engineer_site reliability                      0.355212  0.141757  0.202643   
Scientist                                      0.416667  0.262195  0.321856   
full_stack_.NET                                0.696774  0.624277  0.658537   
full_stack_Java                                0.632667  0.556927  0.592386   
full_stack_.JavaScript                         0.686091  0.704440  0.695144   
full_stack_PHP                                 0.687873  0.643125  0.664747   
full_stack_python                              0.642676  0.548254  0.591722   
back_end_Java                                  0.606329  0.617169  0.611701   
back_end_.JavaScript                           0.457177  0.467685  0.462371   
back_end_.NET                                  0.524378  0.559851  0.541534   
back_end_C++                                   0.548303  0.438321  0.487182   
back_end_python                                0.560851  0.453049  0.501219   
back_end_PHP                                   0.534188  0.492350  0.512416   
Mean                                           0.554993  0.416655  0.461897   

                                                             test            \
                                               accuracy precision    recall   
Academic researcher                            0.965070  0.398810  0.202417   
Blockchain                                     0.985845  0.363636  0.223602   
Cloud infrastructure engineer                  0.943626  0.353571  0.200405   
Data or business analyst                       0.968194  0.370370  0.131579   
Data scientist or machine learning specialist  0.965368  0.656863  0.457859   
DevOps specialist                              0.937503  0.472103  0.179445   
Developer_back-end                             0.703134  0.578693  0.615301   
Developer_desktop or enterprise applications   0.897516  0.649231  0.194829   
Developer_embedded applications or devices     0.962740  0.625616  0.292627   
Developer_front-end                            0.833829  0.582219  0.364892   
Developer_full-stack                           0.743170  0.676215  0.731455   
Developer_game or graphics                     0.980713  0.375000  0.272727   
Developer_mobile                               0.943899  0.736626  0.585605   
Engineer_data                                  0.960905  0.572519  0.196850   
Engineer_site reliability                      0.982052  0.243243  0.107143   
Scientist                                      0.977540  0.257353  0.192308   
full_stack_.NET                                0.900045  0.703312  0.628463   
full_stack_Java                                0.876841  0.642254  0.544153   
full_stack_.JavaScript                         0.855099  0.691596  0.700041   
full_stack_PHP      

In [20]:
original_features = original_df.drop('DevType',level=0, axis=1).droplevel(level=0,axis=1).columns
original_modeling_utils.save_results(all_classification_report=classification_report_original_features,
                                     model_name='Logistic-Regression_Original-Features_1.0.0',
                                     data_path=LOAD_SKILLS_DEV, features=original_features)


clustered_features = clustered_df.drop('DevType',level=0, axis=1).droplevel(level=0,axis=1).columns
clustered_modeling_utils.save_results(all_classification_report=classification_report_clustered_features,
                                     model_name='Logistic-Regression_Clustered-Features_1.0.0',
                                     data_path=LOAD_SKILLS_DEV, features=clustered_features)

2023/08/17 12:25:36 INFO mlflow.tracking.fluent: Experiment with name 'skills_rec_analysis' does not exist. Creating a new experiment.


In [21]:
MlflowUtils.get_runs()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,metrics.recall,metrics.f1_score,metrics.precision,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.runName,tags.mlflow.source.name
0,b4928cc3e3bf4f21b8b9138310447d2c,735982700926372623,FINISHED,file:///D:/protofolio_projects/Dev_skills_reco...,2023-08-17 09:25:36.899000+00:00,2023-08-17 09:25:36.934000+00:00,0.894298,0.324855,0.378904,0.522859,mahmo,LOCAL,Logistic-Regression_Clustered-Features_1.0.0,C:\Users\mahmo\anaconda3\envs\Rec-skills\lib\s...
1,91321ce64e8646d7a8e28dac0a5d8deb,735982700926372623,FINISHED,file:///D:/protofolio_projects/Dev_skills_reco...,2023-08-17 09:25:36.843000+00:00,2023-08-17 09:25:36.877000+00:00,0.897505,0.404598,0.446315,0.531665,mahmo,LOCAL,Logistic-Regression_Original-Features_1.0.0,C:\Users\mahmo\anaconda3\envs\Rec-skills\lib\s...


### as we see from the above dataframe, original features outperform clustered features in all metricises especially recall & f1_score that we care more
#### 1. so will use the original features not all the clustered features
#### 2. And we will use These metricises as reference for any further complex model